In [524]:
import random
import math
import numpy as np

class Node:
  def __init__(self, state, parent=None, name='child'):
    self.name = name
    self.state = state
    self.terminal = self.state.is_terminal()
    self.is_fully_expanded = self.terminal
    self.parent = parent
    self.children = {}
    self.visits = 0
    self.reward = 0

  def __str__(self) -> str:
    return str(self.state)

class MCTS:
  def search(self, state, iterations=100):
    self.root = Node(state, name="root")

    for _ in range(iterations):
      # Selection and Expansion
      node = self.select(self.root)
      # print(node.state)
      # Simulation
      winner = self.rollout(node.state)
      # print(f'The winner is {winner}')
      # Backpropagation
      self.backpropagate(node, winner)
    
    for child in self.root.children.values():
      print(self.root.visits, child.visits, child.reward)

    # Return the best child of the root node
    return self.best_child(self.root)

  def select(self, node):
    # Select the best child until a leaf node is reached
    while not node.terminal:
      if not node.is_fully_expanded:
        # print("Not fully expanded")
        return self.expand(node)
      else:
        # print("Fully expanded")
        node = self.best_child(node)
    return node

  def expand(self, node):
    # Expand the node by adding a new child
    valid_moves = node.state.get_valid_moves()
    for move in valid_moves:
      # print(node.children, f'{node.state.active_index} : {move}')
      if(f'{node.state.active_index} : {move}' not in node.children):
        # print("OK")
        new_state = node.state.make_move(move)
        # print(f'Player token is : {new_state.player.token}')
        new_node = Node(new_state, node)
        node.children[f'{node.state.active_index} : {move}'] = new_node

        if len(valid_moves) == len(node.children):
          node.is_fully_expanded = True
        return new_node

    #Debugging
    print("Not good if I arrived here")

  #To verify#####
  def rollout(self, state):
    # Play the game to completion by making random moves
    # while np.any(state.get_valid_moves()):
    while not state.is_terminal():
      state = state.make_random_move()
    return state.get_winner()

  def backpropagate(self, node, winner):
    # Update the node and its ancestors with the result of the simulation
    while node is not None:
      node.visits += 1
      node.reward += winner
      node = node.parent

  # To verify #####
  def best_child(self, node, c_param=1.4):
    # Use the UCB formula to select the next child to explore
    best_score = -float("inf")
    best_child = None
    for child in node.children.values():
      if child.state.player.token == 'X': current_player = -1
      elif child.state.player.token == 'O': current_player = 1
      score = current_player*child.reward / child.visits + c_param * math.sqrt(math.log(node.visits) / child.visits)
      if score > best_score:
        best_score = score
        best_child = child

    # if(not best_child.terminal):
    #   print(f'This is the best child "{best_child}"')
    return best_child

In [553]:
import numpy as np
import random

N = 9                           # Tic Tac Toe size of 3x3
three_values = [3, 21]          # Triple 1 or triple 7

class StatePlayer:
    def __init__(self, player=None):
        self.token = "X"
        self.token_value = 1
        if(player is not None):
            self.token = player.token
            self.token_value = player.token_value

    def change_player(self):
        if(self.token=="X"):
            self.token = "O"
            self.token_value = 7
        else:
            self.token = "X"
            self.token_value = 1

class State():
    def __init__(self, matrix, player):
        self.matrix = matrix
        self.active_index = 1
        self.player = player

    def get_valid_moves(self):
        return np.where(self.matrix==0)[0]

    # Modified
    def make_move(self, move):
        new_player = StatePlayer(self.player)
        board = State(self.matrix.copy(), new_player)
        board.matrix[move] = board.player.token_value
        board.player.change_player()
        return board

    def make_random_move(self):
        valid_moves = self.get_valid_moves()
        move = random.choice(valid_moves)
        # new_player = StatePlayer()
        # new_player.token = self.player.token
        # new_player.token_value = self.player.token_value
        return self.make_move(move)

    def get_winner(self):
        matrix2D = self.matrix.reshape((3,3))
        
        diagonal = np.sum(np.diagonal(matrix2D))
        opposite_diagonal = np.sum(np.diagonal(np.fliplr(matrix2D)))
        lines = np.sum(matrix2D, axis=1)
        columns = np.sum(matrix2D, axis=0)

        if three_values[0]==diagonal or three_values[0]==opposite_diagonal or three_values[0] in lines or three_values[0] in columns:
            # print(f'##{matrix2D}##')
            return 1
        elif three_values[1]==diagonal or three_values[1]==opposite_diagonal or three_values[1] in lines or three_values[1] in columns:
            return -1
        elif(not np.any(self.matrix==0)):
            return 0
        else:
            return None

    def is_terminal(self):
        return self.get_winner() is not None

    def __str__(self) -> str:
        board = ''
        for i, cell in enumerate(self.matrix):
            if(i%3==0):
                board += '\n'
            board += str(cell) + ' '
        
        return board


player = StatePlayer()
current_state = State(np.array([7,0,7,
                                0,1,0,
                                0,0,0]), player)
tree = MCTS()
new_board = tree.search(current_state, 1000)
print(new_board)

1000 899 56
1000 23 -14
1000 21 -13
1000 23 -14
1000 21 -13
1000 13 -11

7 1 7 
0 1 0 
0 0 0 
